In [1]:
import camelot
import pandas as pd
import numpy as np
import re
import os

In [2]:
# Helper codes

def clean_country_name(name):
    name = str(name).strip()              
    name = re.sub(r'\s+', ' ', name)     
    name = re.sub(r"\s*-\s*.*", "", name)
    name = re.sub(r"[^A-Za-z\s]", "", name)
    name = re.sub(r'[^A-Za-z\s]', '', name)  
    return name.title()                   

# --- Constants ---
MONTH_ORDER = [
    "January", "February", "March", "April", "May", "June",
    "July", "August", "September", "October", "November", "December"
]

# --- Helpers ---
def get_months(df):
    """Return ordered list of existing month columns."""
    return [m for m in MONTH_ORDER if m in df.columns]


In [3]:
exclude_words = [
    "A S I A",
    "ASEAN",
    "SUB-TOTAL",
    "EAST ASIA",
    "SOUTH ASIA",
    "MIDDLE EAST",
    "A M E R I C A",
    "NORTH AMERICA",
    "SOUTH AMERICA",
    "E U R O P E",
    "WESTERN EUROPE",
    "NORTHERN EUROPE",
    "SOUTHERN EUROPE",
    "EASTERN EUROPE",
    "EASTERN MEDITERRANEAN EUROPE",
    "AUSTRALASIA/PACIFIC",
    "TOTAL (CIS & RUSSIA)"
    "A F R I C A	"
]

In [4]:
# Read tables from PDF (PAGE 1)
pdf_file = "/Users/kim/Desktop/repos/Philippines_Visitor/data/rawData/2016-JAN-DEC.pdf"
tables = camelot.read_pdf(pdf_file, pages='1', flavor="stream")

# Combine all extracted tables into one DataFrame
df1 = pd.concat([table.df for table in tables], ignore_index=True)

# Clean data 
df1 = df1.iloc[2:].reset_index(drop=True) # Drop top 4 rows (not needed for the table)
df1.columns = range(df1.shape[1])

df1.drop(
    df1[df1[0].str.upper().isin(exclude_words)].index,
    inplace=True
)

df1 = df1.reset_index(drop=True).iloc[:-1]


In [5]:
# Read tables from PDF (PAGE 2)
tables = camelot.read_pdf(pdf_file, pages='2', flavor="stream")

# Combine all extracted tables into one DataFrame
df2 = pd.concat([table.df for table in tables], ignore_index=True)

# Clean data 
df2 = df2.iloc[3:].reset_index(drop=True) # Drop top 4 rows (not needed for the table)
df2.columns = range(df2.shape[1])

df2.drop(
   df2[df2[0].str.upper().isin(exclude_words)].index,
    inplace=True
)

df2 = df2[df2[0] != 'EASTERN MEDITERRANEAN EUROPE**'].reset_index(drop=True)

df2 = df2.reset_index(drop=True)

# DROP TOTAL RUSSIAN FEDERATION 
df2 = df2.reset_index(drop=True)
df2 = df2.drop(26).reset_index(drop=True)
df2 = df2.drop(23).reset_index(drop=True)

df2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,ARGENTINA,392,338,310,243,191,116,114,105,100,141,161,158,"2,369",0.04,"1,745",35.76
1,BRAZIL,412,422,346,319,317,313,274,249,317,415,541,595,"4,520",0.08,"3,495",29.33
2,COLOMBIA,135,81,113,84,100,129,97,93,153,170,139,137,"1,431",0.02,"1,228",16.53
3,PERU,67,40,58,44,44,52,35,35,39,60,65,42,581,0.01,561,3.57
4,VENEZUELA,26,24,32,26,21,25,32,20,22,15,19,9,271,0.00,221,22.62
5,ANDORRA,19,23,28,28,25,24,36,48,39,51,21,22,364,0.01,329,10.64
6,AUSTRIA,"1,716","1,349","1,257",998,705,623,"1,277",914,608,828,"1,007","1,805","13,087",0.22,"12,267",6.68
7,BELGIUM,"1,151","1,266","1,635","1,025",964,"1,071","1,956",782,856,"1,064","1,091","1,616","14,477",0.24,"12,825",12.88
8,FRANCE,"5,356","7,146","5,915","5,563","4,356","2,809","4,233","3,325","2,360","3,905","4,348","6,068","55,384",0.93,"45,505",21.71
9,GERMANY,"8,337","9,869","10,415","7,018","5,901","4,082","6,112","5,814","4,779","6,598","7,357","10,081","86,363",1.45,"75,348",14.62


In [6]:
# Read tables from PDF (PAGE 3)
tables = camelot.read_pdf(pdf_file, pages='3', flavor="stream")

# Combine all extracted tables into one DataFrame
df3 = pd.concat([table.df for table in tables], ignore_index=True)

# Clean data 
df3 = df3.iloc[2:].reset_index(drop=True) # Drop top 4 rows (not needed for the table)
df3.columns = range(df3.shape[1])

df3.drop(
   df3[df3[0].str.upper().isin(exclude_words)].index,
    inplace=True
)
# DROP EMPTY ROW
df3 = df3.drop([7,11]).reset_index(drop=True)

# DROP UNWANTED ROWS AT THE END 
df3 = df3.iloc[:-20]
df3

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,AUSTRALIA,"25,274","18,438","23,553","21,231","19,886","17,827","19,232","15,575","17,584","20,220","19,618","32,660","251,098",4.21,"241,187",4.11
1,GUAM,"2,485","2,430","3,089","3,095","3,792","4,629","4,688","3,108","2,670","2,675","2,880","3,236","38,777",0.65,"35,262",9.97
2,NAURU,2,,1,-,2,3,-,4,-,5,-,-,17,0.00,20,-15.00
3,NEW ZEALAND,"2,011","1,283","1,724","1,991","1,774","1,840","1,975","1,535","1,621","1,829","2,143","3,705","23,431",0.39,"20,579",13.86
4,PAPUA NEW GUINEA,568,537,553,589,755,756,675,567,822,603,564,749,"7,738",0.13,"5,910",30.93
5,NIGERIA,163,90,105,129,187,164,144,141,108,109,83,66,"1,489",0.02,"1,430",4.13
6,SOUTH AFRICA,435,350,559,436,361,362,426,359,345,353,409,722,"5,117",0.09,"4,570",11.97
7,RESIDENCES,"6,679","6,044","5,651","5,098","4,126","4,151","4,269","3,424","3,379","3,999","5,409","4,997","57,226",0.96,"49,074",16.61
8,T O T A L,"525,832","537,194","485,892","452,396","427,934","444,043","541,298","490,190","414,202","432,157","472,258","553,186","5,776,582",96.81,"5,148,819",12.19
9,OVERSEAS FILIPINOS***,"16,426","12,531","24,378","19,202","17,515","15,095","19,574","12,549","8,741","10,868","10,092","23,452","190,423",3.19,"211,863",-10.12


In [7]:
combined_df = pd.concat([df1, df2, df3], ignore_index=True)

combined_df.columns = [
        "Country",
        "January",
        "February",
        "March",
        "April",
        "May",
        "June",
        "July",
        "August",
        "September",
        "October",
        "November",
        "December",
        "Total",
        "Percentage",
        "Previous Total",
        "Growth Rate"
        ]

# Clean and convert column to float

combined_df['Total'] = (
    combined_df['Total']
    .astype(str)               # ensure everything is string
    .str.replace(',', '')      # remove commas
    .str.strip()               # remove leading/trailing spaces
    .replace({'': np.nan, '-': np.nan})  # replace empty strings and '-' with NaN
    .astype(float)             # convert to float
)
combined_df = combined_df.sort_values(by="Total", ascending=False).reset_index(drop=True)

# Copy the row you want to move
row_to_move = combined_df.iloc[7]

# Drop the original row
combined_df = combined_df.drop(7).reset_index(drop=True)

top = combined_df.iloc[:1]   
bottom = combined_df.iloc[1:] 

# Insert the row
combined_df = pd.concat([top, pd.DataFrame([row_to_move]), bottom]).reset_index(drop=True)

combined_df['Country'] = combined_df['Country'].apply(clean_country_name)

combined_df['Country'] = combined_df['Country'].replace('Independent States', 'Commonwealth of Independent States')
combined_df['Country'] = combined_df['Country'].replace('Hongkong', 'Hong Kong')
combined_df['Country'] = combined_df['Country'].replace('Korea', 'South Korea')



combined_df.to_csv("/Users/kim/Desktop/repos/Philippines_Visitor/data/cleanedData/2016cleanData.csv")
 

In [8]:
# Store Monthly Data
monthly_folder = "/Users/kim/Desktop/repos/Philippines_Visitor/data/monthly_visitors"

monthly_df = combined_df.iloc[2:].reset_index(drop=True) 
monthly_df['Country'] = monthly_df['Country'].apply(clean_country_name)

months = get_months(combined_df)

monthly_df = monthly_df[['Country'] + months]

mon_out_path = os.path.join(monthly_folder, f"2016_monthly.csv")
monthly_df.to_csv(mon_out_path, index=False)

print(f"Saved 2016 Monthly Visitors csv file")


Saved 2016 Monthly Visitors csv file


In [9]:
# Store Special Categories Data

cat_folder = "/Users/kim/Desktop/repos/Philippines_Visitor/data/special_category"

cat_df = combined_df.head(2)
cat_df['Country'] = cat_df['Country'].replace("T O T A L", "TOTAL")
cat_df['Country'] = cat_df['Country'].replace("Overseas Filipinos***", "Overseas Filipinos")
cat_df["Country"] = cat_df["Country"].astype(str).str.strip().str.title()
cat_df["Country"] = cat_df["Country"].apply(clean_country_name)

cat_df = cat_df.iloc[:, :-4]

cat_out_path = os.path.join(cat_folder, f"2016_category.csv")
cat_df.to_csv(cat_out_path, index=False)

print(f"Saved 2016 Category csv file")

Saved 2016 Category csv file


/var/folders/fj/1bp6c_7s3b51sm954hmfznt40000gn/T/ipykernel_64359/3866474738.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_df['Country'] = cat_df['Country'].replace("T O T A L", "TOTAL")
/var/folders/fj/1bp6c_7s3b51sm954hmfznt40000gn/T/ipykernel_64359/3866474738.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_df['Country'] = cat_df['Country'].replace("Overseas Filipinos***", "Overseas Filipinos")
/var/folders/fj/1bp6c_7s3b51sm954hmfznt40000gn/T/ipykernel_64359/3866474738.py:8: SettingWithCo

In [10]:
year_folder = "/Users/kim/Desktop/repos/Philippines_Visitor/data/yearly_total"

year_df = combined_df[['Country', 'Total', 'Percentage', 'Previous Total', 'Growth Rate']]
year_df = year_df.iloc[2:]

year_df["Previous Total"] = (
    year_df["Previous Total"]
        .astype(str)
        .str.replace('"', '', regex=False)
        .str.replace(',', '', regex=False)   # REMOVE COMMAS
        .str.strip()
)

year_df["Previous Total"] = year_df["Previous Total"].astype(float)

year_df["Growth Rate"] = pd.to_numeric(year_df["Growth Rate"], errors = 'coerce')
year_df["Percentage"] = pd.to_numeric(year_df["Percentage"], errors = 'coerce')

year_path = os.path.join(year_folder, f"2016_year.csv")
year_df.to_csv(year_path, index=False)

print(f"Saved 2016 Year csv file")


Saved 2016 Year csv file
